##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Flower classification with TensorFlow Lite Model Maker with TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">      
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes this Model Maker library to illustrate the adaption and conversion of a commonly-used image classification model to classify flowers on a mobile device.

## Prerequisites

To run this example, we first need to install serveral required packages, including Model Maker package that in github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [1]:
!pip install tflite-model-maker

     |████████████████████████████████| 591 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 36.0 MB/s 
     |████████████████████████████████| 6.3 MB 42.4 MB/s 
     |████████████████████████████████| 840 kB 47.9 MB/s 
     |████████████████████████████████| 1.1 MB 47.0 MB/s 
     |████████████████████████████████| 120 kB 51.9 MB/s 
     |████████████████████████████████| 213 kB 50.8 MB/s 
     |████████████████████████████████| 87 kB 6.4 MB/s 
     |████████████████████████████████| 103 kB 49.5 MB/s 
     |████████████████████████████████| 1.2 MB 52.5 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |████████████████████████████████| 596 kB 46.9 MB/s 
     |████████████████████████████████| 47.6 MB 36 kB/s 
     |████████████████████████████████| 99 kB 9.2 MB/s 
     |████████████████████████████████| 352 kB 51.4 MB/s 
     |████████████████████████████████| 208 kB 26.3 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 s

Import the required packages.

In [18]:
import numpy as np
import zipfile
import tensorflow as tf
from tflite_model_maker.config import ExportFormat
assert tf.__version__.startswith('2')
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

## Simple End-to-End Example

### Get the data path

Let's get some images to play with this simple end-to-end example. Hundreds of images is a good start for Model Maker while more data could achieve better accuracy.

In [10]:
zip_ref = zipfile.ZipFile('/content/train.zip', 'r')
zip_ref.extractall()
zip_ref.close()

You could replace `image_path` with your own image folders. As for uploading data to colab, you could find the upload button in the left sidebar shown in the image below with the red rectangle. Just have a try to upload a zip file and unzip it. The root file path is the current path.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_image_classification.png" alt="Upload File" width="800" hspace="100">

If you prefer not to upload your images to the cloud, you could try to run the library locally following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) in github.

### Run the example
The example just consists of 4 lines of code as shown below, each of which representing one step of the overall process.


1.   Load input data specific to an on-device ML app. Split it to training data and testing data.

In [12]:
data = DataLoader.from_folder('train')
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 16854, num_label: 33, labels: Apple Braeburn, Apple Granny Smith, Apricot, Avocado, Banana, Blueberry, Cactus fruit, Cantaloupe, Cherry, Clementine, Corn, Cucumber Ripe, Grape Blue, Kiwi, Lemon, Limes, Mango, Onion White, Orange, Papaya, Passion Fruit, Peach, Pear, Pepper Green, Pepper Red, Pineapple, Plum, Pomegranate, Potato Red, Raspberry, Strawberry, Tomato, Watermelon.


2. Customize the TensorFlow model.

In [13]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 33)                42273     
Total params: 3,455,297
Trainable params: 42,273
Non-trainable params: 3,413,024
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5
474/474 [==============================] - 73s 87ms/step - loss: 1.0812 - accuracy: 0.9262
Epoch 2/5
474/474 [==============================] - 41s 87ms/step - loss: 0.7802 - accuracy: 0.9982
Epoch 3/5
474/474 [==============================] - 42s 88ms/step - loss: 0.7573 - accuracy: 0.9995
Epoch 4/5
474/474 [==============================] - 41s 88ms/step - loss: 0.7451 - accuracy: 0.9998
Epoch 5/5
474/474 [==============================] - 42s 88ms/step - loss: 0.7366 - accuracy: 0.9998


3. Evaluate the model.

In [14]:
loss, accuracy = model.evaluate(test_data)

53/53 [==============================] - 10s 95ms/step - loss: 0.7077 - accuracy: 1.0000


config = QuantizationConfig.for_float16()4.  Export to TensorFlow Lite model.
You could download it in the left sidebar same as the uploading part for your own use.

In [19]:
config = QuantizationConfig.for_float16()
model.export(export_dir='/content/', tflite_filename='model_fp16.tflite', quantization_config=config)
model.export(export_dir='/content/', export_format=ExportFormat.LABEL)

INFO:tensorflow:Assets written to: /tmp/tmpet21sepr/assets


INFO:tensorflow:Assets written to: /tmp/tmpet21sepr/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmplwetl2mc/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmplwetl2mc/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: /content/model_fp16.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: /content/model_fp16.tflite


INFO:tensorflow:Saving labels in /content/labels.txt


INFO:tensorflow:Saving labels in /content/labels.txt


After this simple 4 steps, we can now download the model and label files, and continue to the next step in the [codelab](https://codelabs.developers.google.com/codelabs/recognize-flowers-with-tensorflow-on-android/#4).

For a more comprehensive guide to TFLite Model Maker, please refer to this [notebook](https://colab.sandbox.google.com/github/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/demo/image_classification.ipynb) and its [documentation](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).
